# Datenbereinigung

In [ ]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
import matplotlib.pyplot as plt
import json

In [ ]:
#mapbox token definition
mapbox_token = "pk.eyJ1IjoicHl0aG9ucyIsImEiOiJja2s2dzhmMjcwODM3MndueGR5Y3E4b2QxIn0.09n-pctdAFhEa4NrPbE6MA"
px.set_mapbox_access_token(mapbox_token)

In [ ]:

#twitter account data
df = pd.read_csv("data/accounts_data.csv")
df = df.drop(columns = ["Unnamed: 0"])

#wahlkreise wahlergebnisse
wahl = pd.read_csv("data/wahl.csv", sep = ";", encoding="utf8")

with open("data/geometrie.geojson") as f:
    geojs = json.load(f)
#wahlkreise shapefiles
gj = gpd.read_file("data/btw17/Geometrie_Wahlkreise_19DBT_geo.shp")

#wahlkreise und twitter verbindung
ver = pd.read_csv("data/verbindung.csv", sep = ";")

In [ ]:
#better wahl data
wahl["Namen"] = wahl["Vorname"] + " " + wahl["Name"]
wahl = wahl.drop(columns=["Geburtsjahr", "Beruf", "Wiedergewählt", "Vorname", "Name"])
wahl = wahl.dropna()
wahl = wahl.rename(columns = {"Gewählt_Wahlkreis_Nr": "WKR_NR"} )


In [ ]:
#merging the. shapefiles and the wahlergebnisse
gp = pd.merge(gj, wahl, on="WKR_NR", how = "inner")

In [ ]:
#merging twitter handles and wahlen
gp = pd.merge(ver, gp, on = "Namen", how = "inner")

#creating new columns 
gp["Kreis"] = gp.WKR_NR -1
gp["twitter"] = gp.name_id.notna()
gp["twitter"] = gp.twitter*1

#dropping other cols
gp = gp.drop(columns=["LAND_NR", "Gewählt_Land","WKR_NAME", "Unnamed: 0", "geometry", "LAND_NAME"])

# exploration

In [ ]:
#twitter datei
df.head(2)


In [ ]:
gj.head(2)

In [ ]:
gp.head()

In [ ]:
geojs["features"][0]["properties"]

In [ ]:
fig = px.choropleth_mapbox(
    gp, geojson = geojs, featureidkey = "properties.WKR_NR", locations = gp.WKR_NR, 
    color = gp.Partei_KurzBez,hover_name = gp.name_id
    )

fig.update_layout(mapbox = {
    "center": {"lon": 10.447 , "lat": 51.1633},
    "zoom": 4

    })
fig.show()

In [ ]:
 namen1 = gp.name_id
 namen2 = df.name_id

In [ ]:
len(set(namen1).intersection(set(namen2)))

In [ ]:
set(namen1).difference(set(namen2))

## Match Aufgaben:
### Wahlkreise Shape und Menschen
### Menschen und twitter Profil

In [ ]:
df[df.name_id == "@ninawarken"]